# Goal - Create conversation with artifacts
- Artifacts are presented in the system message (along with their history eventually)
- Artifacts are referenced in the conversation by their nickname
- Artifacts can be edited by the AI by rewriting their content (need to use Claude so that I can present this with XML tags)
- Users can modify artifacts directly and the changes are reflected in the conversation
- Demo CRUD
  - User creates artifact by requesting it - user deletes artifact
  - User retrieves artifact by requesting it
- Demo relationship between artifacts
- Demo anchors in artifacts?

## Scenario
- realestate agent is gathering information about a property (might use tools to get more information)
- they talk about the property
- they make a template email - which generates a new artifact
- they edit the email (making references to other artifacts)
- they send it

## Unknown
- [ ] How do I have the assistant generate an artifact?


In [43]:

import anthropic
import json

class Artifact:
    def __init__(self, identifier, type, title, content):
        self.identifier = identifier
        self.type = type
        self.title = title
        self.content = content

    def __str__(self):
        return f"<artifact identifier={self.identifier} type={self.type} title={self.title}>\n{self.content}\n</artifact>"
    
class Tool:
    def __init__(self, schema, callable):
        self.schema = schema
        self.callable = callable
        self.name = schema["name"]

class Conversation:
    def __init__(self, artifacts=None, tools=None,filename=None):
        self.client = anthropic.Anthropic()
        self.model = "claude-3-sonnet-20240229"
        self.messages = []
        self.artifacts = artifacts or []
        self.tools = tools or []
        self.filename = None
        if filename:
            self.filename = filename
            try:
                with open(filename, "r") as file:
                    self.messages = json.load(file)
            except FileNotFoundError:
                print(f"File {filename} not found")
        
    def say(self, message):
        self.messages.append(
            {
                "role": "user", 
                "content": message
            }
        )
        system_message = self._generate_system_message()
        tools = [t.schema for t in self.tools]
        response = self.client.messages.create(
            model=self.model,
            system=system_message,
            messages=self.messages,
            max_tokens=3000,
            temperature=0.7,
        )

        # Handle potential tool use
        assistant_messages = []
        while response.stop_reason == "tool_use":
            print("\n"+str(response.content))

            tool_result_messages  = []
            for block in response.content:
                if block.type != "tool_use":
                    # TODO: this could be improved. As is we're going to just concatenate the text of all the blocks.
                    # This might not make sense to the reader since the tool calls are missing and the assistant might refer to them.
                    # An improvement would be to have the assistant say something like "I used the following tools to answer the question: ..."
                    # and then list the tools used and their results.
                    assistant_messages.append(block.text)
                else:
                    tool_use = block
                    tool_name = tool_use.name
                    tool_input = tool_use.input
                    tool_result = self._process_tool_call(tool_name, tool_input)
                    tool_result_messages.append({
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": tool_result,
                    })
            
            self.messages.append({"role": "assistant", "content": response.content})
            self.messages.append({
                "role": "user",
                "content": tool_result_messages,
            })
            
            # Get final response after tool use
            response = self.client.messages.create(
                model=self.model,
                system=system_message,
                messages=self.messages,
                max_tokens=3000,
                temperature=0.7,
                tools=tools,
            )
        
        assistant_messages.append(response.content[0].text)
        assistant_message = "\n".join(assistant_messages)
        self.messages.append({"role": "assistant", "content": assistant_message})
        if self.filename:
            self._save()
        print(assistant_message)
        return assistant_message
    
    def _process_tool_call(self, tool_name, tool_input):
        for tool in self.tools:
            if tool.name == tool_name:
                return tool.callable(tool_input)
        raise Exception(f"Tool {tool_name} not found")
    
    def _generate_system_message(self):
        artifacts_info = "\n".join([str(artifact) for artifact in self.artifacts])
        system_message = f"""\
You are a helpful assistant.

<artifacts_info>
Artifacts are self-contained pieces of content that can be referenced in the conversation. The assistant can generate artifacts during the course of the conversation upon request of the user. Artifacts have the following format:

```
<artifact identifier="hexidecimal-hash" type="mime_type" title="title">
...actual content of the artifact...
</artifact>
```

example identifiers: 18bacG4a, 3baf9f83, 98acb34d
example types: text/markdown, text/plain, application/json, image/svg+xml
example titles: "Simple Python factorial script", "Blue circle SVG", "Metrics dashboard React component"

<artifact_instructions>
- The user has access to the artifacts, therefore the assistant should not reproduct the full content of the artifact in the conversation, but should instead reference it by its `identifier` using an anchor tag like this: `<a href="#18bacG4a">linked text</a>`.
- The linked text should make sense in the context of the conversation. The assistant must supply the linked text.
- The user can similarly refer to the artifacts via an anchor. But they can also just say "the thing we were discussing earlier".
- Artifact titles must be short, descriptive, and unique.
- All existing artifacts are presented in the <artifacts> tag below.
</artifact_instructions>

</artifacts_info>

<artifacts>
{artifacts_info}
</artifacts>
"""
        return system_message

    def _save(self):
        if self.filename:
            with open(self.filename, "w") as file:
                messages_to_save = []
                for message in self.messages:
                    if isinstance(message['content'], list):
                        # Handle list of content blocks
                        new_content = []
                        for block in message['content']:
                            if hasattr(block, 'dict'):
                                new_content.append(block.dict())
                            else:
                                new_content.append(block)
                        messages_to_save.append({
                            'role': message['role'],
                            'content': new_content
                        })
                    else:
                        # Content is a simple string or already a dict
                        messages_to_save.append(message)
                json.dump(messages_to_save, file)





In [44]:
artifacts = [
    Artifact(
        identifier="18bacG4a",
        type="application/json", 
        title="742 Maple Street Listing", 
        content="""\
{
    "address": "742 Maple Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
    "last_sold": {
        "date": "2019-06-15",
        "price": 172000
    },
    "property_tax": 2800,
    "zestimate": 189500
}
"""),
    Artifact(
        identifier="3baf9f83", 
        type="application/json", 
        title="742 Maple Street Comparables", 
        content="""\
[
    {
        "address": "738 Maple Street",
        "city": "Cedar Rapids", 
        "state": "IA",
        "price": 179900,
        "beds": 3,
        "baths": 2,
        "sqft": 1400,
        "year_built": 1975,
        "last_sold": {
            "date": "2020-03-15",
            "price": 165000
        },
        "distance_miles": 0.1,
        "zestimate": 183000
    },
    {
        "address": "755 Oak Drive",
        "city": "Cedar Rapids",
        "state": "IA", 
        "price": 192000,
        "beds": 3,
        "baths": 2.5,
        "sqft": 1500,
        "year_built": 1980,
        "last_sold": {
            "date": "2021-08-01",
            "price": 180000
        },
        "distance_miles": 0.3,
        "zestimate": 195000
    },
    {
        "address": "729 Elm Court",
        "city": "Cedar Rapids",
        "state": "IA",
        "price": 187500,
        "beds": 3,
        "baths": 2,
        "sqft": 1425,
        "year_built": 1977,
        "last_sold": {
            "date": "2020-11-30",
            "price": 175000
        },
        "distance_miles": 0.2,
        "zestimate": 191000
    }
]
"""),
    Artifact(
        identifier="98acb34d", 
        type="text/plain", 
        title="Prospective Buyer Listing Email Template", 
        content="""
Dear {buyer_name},

I wanted to bring to your attention an exciting new listing at {address}, {city}, {state} that I believe would be perfect for you.

This beautiful {beds} bedroom, {baths} bathroom home offers {sqft} square feet of living space and was built in {year_built}. It is currently listed at ${price:,}, which represents excellent value for this desirable neighborhood.

Some key features that make this property stand out:
- Spacious layout with {beds} bedrooms
- {sqft} square feet of living space
- Well-maintained home built in {year_built}
- Current Zestimate: ${zestimate:,}

To give you some context about the local market, there are several comparable properties in the immediate vicinity:
- A similar {beds} bed/{baths} bath home just {distance_miles} miles away recently sold for ${last_sold[price]:,}
- Nearby properties range from ${price:,} to ${price:,} in this area
- Most homes in this neighborhood were built in the 1970s-1980s

Would you like to schedule a viewing of this property? I have several time slots available this week and would be happy to show you around.

Best regards,
Your Real Estate Agent
"""
    ),
]



In [45]:
c = Conversation(artifacts=artifacts)


In [46]:
c.say("Tell me about the artifacts you have access to");


I have access to three artifacts:

1. <a href="#18bacG4a">A JSON listing for 742 Maple Street</a> - This contains details like the address, price, number of bedrooms/bathrooms, square footage, year built, description, features, last sale information, property tax estimate, and Zestimate.

2. <a href="#3baf9f83">Comparable home listings near 742 Maple Street</a> - This is a JSON array with 3 other listings that are geographically close to 742 Maple Street, including their addresses, prices, details, last sale data, distance from 742 Maple, and Zestimate.

3. <a href="#98acb34d">A text template for emailing prospective buyers</a> - This is a plain text template that can be populated with details from a listing to create an email telling a prospective buyer about the property.

The artifacts provide comprehensive information about a specific real estate listing at 742 Maple Street, data on comparable nearby listings to gauge market value, and a template for marketing the listing to potent

In [37]:
c.say("Can you fill in the blanks in the email template?");

Here is the email template with the placeholders filled in using data from the <a href="#18bacG4a">742 Maple Street listing</a>:

<artifact identifier=98acb34d type=text/plain title="Prospective Buyer Listing Email Template">
Dear {buyer_name},

I wanted to bring to your attention an exciting new listing at 742 Maple Street, Cedar Rapids, IA that I believe would be perfect for you.

This beautiful 3 bedroom, 2 bathroom home offers 1,450 square feet of living space and was built in 1978. It is currently listed at $185,000, which represents excellent value for this desirable neighborhood.  

Some key features that make this property stand out:
- Spacious layout with 3 bedrooms
- 1,450 square feet of living space
- Well-maintained home built in 1978
- Current Zestimate: $189,500

To give you some context about the local market, there are several comparable properties in the immediate vicinity:
- A similar 3 bed/2 bath home just 0.1 miles away recently sold for $165,000  
- Nearby properti

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");

In [ ]:
c.say(");